<a href="https://colab.research.google.com/github/Debugger-Dev/WebScrapping/blob/main/RottenTomatos.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [55]:
import requests
from bs4 import BeautifulSoup
import pandas as pd

In [2]:
url='https://editorial.rottentomatoes.com/guide/140-essential-action-movies-to-watch-now/'
response=requests.get(url)
response

<Response [200]>

In [3]:
html=response.content

In [4]:
soup=BeautifulSoup(html,'lxml')
with open('Rotten tomatos action movies_lxml.html','wb') as file:
  file.write(soup.prettify('utf-8'))

In [ ]:
divs=soup.find_all('div',{'class':'col-sm-18 col-full-xs countdown-item-content'})
divs[0]

In [57]:
headings=[div.find('h2') for div in divs]
movie_names=[heading.find('a').string for heading in headings]
years=[heading.find('span',class_='subtle start-year').string for heading in headings]
years=[year.strip("()") for year in years]
years_final=[int(year) for year in years]
scores=[heading.find('span',class_='tMeterScore').string for heading in headings]
scores=[score.strip("%") for score in scores]
scores_final=[int(score) for score in scores]

In [ ]:
#Way1
consensus=[div.find("div",{'class':'info critics-consensus'}) for div in divs]
common_phrase='Critics Consensus:'
con_len=len(common_phrase)
consensus_text=[con.text[con_len:] if con.text.startswith(common_phrase) else con.text for con in consensus]
consensus_text


In [ ]:
consensus[0].contents

In [ ]:
#Way2
consensus_txt=[con.contents[1].strip() for con in consensus]
consensus_txt

In [10]:
directors=[div.find('div',class_='info director') for div in divs]
#[director.find("a") for director in directors]
#to deal if None va;ue present
final_directors=[None if director.find("a") is None else director.find("a").string for director in directors]

In [11]:
cast_info=[div.find("div",class_='info cast') for div in divs]
cast_info[0]

<div class="info cast">
<span class="descriptor">Starring:</span> <a class="" href="//www.rottentomatoes.com/celebrity/gregory_hines">Gregory Hines</a>, <a class="" href="//www.rottentomatoes.com/celebrity/billy_crystal">Billy Crystal</a>, <a class="" href="//www.rottentomatoes.com/celebrity/jimmy_smits">Jimmy Smits</a>, <a class="" href="//www.rottentomatoes.com/celebrity/steven_bauer">Steven Bauer</a></div>

In [16]:
cast_links=cast_info[0].find_all('a')
cast_names=[link.string for link in cast_links]
cast=",".join(cast_names)
cast

'Gregory Hines,Billy Crystal,Jimmy Smits,Steven Bauer'

In [ ]:
#same but in a different way
cast=[",".join([link.string for link in c.find_all('a')]) for c in cast_info]
cast

In [29]:
headings[0]

<h2><a href="https://www.rottentomatoes.com/m/1018009-running_scared">Running Scared</a> <span class="subtle start-year">(1986)</span> <span class="icon tiny rotten" title="Rotten"></span> <span class="tMeterScore">59%</span><span class="span-icon-audience"><img class="icon-audience" height="16" src="https://images.fandango.com/cms/assets/cf921970-d359-11ea-a15f-bdf29fa24277--spilled.png" width="16"/></span><span class="tMeterScore">58%</span></h2>

In [45]:
adj_scores=[div.find("div", {"class": "info countdown-adjusted-score"}) for div in divs]
#adj_scores[0].contents[1]
adj_score_clean=[score.contents[1].strip("% ") for score in adj_scores]
adj_score_clean[0]

'53978'

In [54]:
synopsis=[div.find("div",{"class":"info synopsis"}) for div in divs]
#synopsis[2].contents[1]
syn_txt=[syn.contents[1] for syn in synopsis]
syn_txt[0]

' Ray and Danny (Gregory Hines, Billy Crystal) are two Chicago police detectives hot on the trail of drug kingpin Julio...'

In [61]:
movies_info=pd.DataFrame()

movies_info['Movie Title']=movie_names
movies_info['Year']=years_final
movies_info['Score']=scores_final
movies_info['Adjusted Score']=adj_score_clean
movies_info['Director']=final_directors
movies_info['Synopsis']=syn_txt
movies_info['Cast']=cast
movies_info['Consensus']=consensus_text

movies_info

,Movie Title,Year,Score,Adjusted Score,Director,Synopsis,Cast,Consensus
0,Running Scared,1986,59,53978,Peter Hyams,"Ray and Danny (Gregory Hines, Billy Crystal) ...","Gregory Hines,Billy Crystal,Jimmy Smits,Steven...",Running Scared struggles to strike a consiste...
1,Equilibrium,2002,40,42234,Kurt Wimmer,"In a futuristic world, a regime has eliminate...","Christian Bale,Emily Watson,Taye Diggs,Angus M...",Equilibrium is a reheated mishmash of other s...
2,Hero,2002,94,101946,Zhang Yimou,In this visually arresting martial arts epic ...,"Jet Li,Tony Leung Chiu Wai,Maggie Cheung Man-y...",With death-defying action sequences and epic ...
3,Road House,1989,39,41524,Rowdy Herrington,"The Double Deuce is the meanest, loudest and ...","Patrick Swayze,Kelly Lynch,Sam Elliott,Ben Gaz...",Whether Road House is simply bad or so bad it...
4,Unstoppable,2010,87,94027,Tony Scott,"When a massive, unmanned locomotive roars out...","Denzel Washington,Chris Pine,Rosario Dawson,Ke...","As fast, loud, and relentless as the train at..."
...,...,...,...,...,...,...,...,...
135,Hard-Boiled,1992,94,95514,John Woo,A cop who loses his partner in a shoot-out wi...,"Chow Yun-Fat,Bowie Lam,Philip Chan,Tony Leung ...",Boasting impactful action as well as surprisi...
136,The Matrix,1999,88,94170,Andy Wachowski,Neo (Keanu Reeves) believes that Morpheus (La...,"Keanu Reeves,Laurence Fishburne,Carrie-Anne Mo...","Thanks to the Wachowskis' imaginative vision,..."
137,Terminator 2: Judgment Day,1991,93,97813,James Cameron,"In this sequel set eleven years after ""The Te...","Arnold Schwarzenegger,Linda Hamilton,Edward Fu...",T2 features thrilling action sequences and ey...
138,Die Hard,1988,94,98933,John McTiernan,New York City policeman John McClane (Bruce W...,"Bruce Willis,Alan Rickman,Bonnie Bedelia,Regin...",Its many imitators (and sequels) have never c...


In [62]:
#movies_info.to_csv("Movies_info.csv",index=False,header=True)
#movies_info.to_excel("Movies_info.xlsx",index=False,header=True)